### test_gdal
Examples of gdal running both with Python wrapper and as a subprocess.

In [1]:
from osgeo import gdal
import numpy as np
import pandas as pd
import subprocess

# this is one of the std err files Andy made that has too big a bounding box for my rotated island grid
f = 'C:/crs/proj/2021_FloSupp_Release/clipfest/std_err/20190830_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83UTM18_NAVD88_1m_SErr_cog.tif'

#### Test simple versions of subprocess.run()
To see output that would go to stderr and stdout, you need `capture_output=True`  
and to print that in a more readable format, use .decode() to convert from a byte string to character string.
(Uncomment the print statements to see the output).

In [2]:
p = subprocess.run(['pwd'],capture_output=True)
print(p.stdout)
p=subprocess.run(['ls','-la'],capture_output=True)
#print(p.stdout.decode())

b'/c/crs/src/CoreBx\n'


#### Test simple gdal command as subprocess

In [3]:
p = subprocess.run(['gdalInfo',f],capture_output=True)
print(p.stdout.decode())

Driver: GTiff/GeoTIFF
Files: C:/crs/proj/2021_FloSupp_Release/clipfest/std_err/20190830_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83UTM18_NAVD88_1m_SErr_cog.tif
       C:/crs/proj/2021_FloSupp_Release/clipfest/std_err/20190830_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83UTM18_NAVD88_1m_SErr_cog.tif.aux.xml
Size is 27617, 24245
Coordinate System is:
PROJCRS["NAD83(2011) / UTM zone 18N",
    BASEGEOGCRS["NAD83(2011)",
        DATUM["NAD83 (National Spatial Reference System 2011)",
            ELLIPSOID["GRS 1980",6378137,298.257222101,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",6318]],
    CONVERSION["UTM zone 18N",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",-75,
            ANGLEUNIT["degree",0.0174532

#### Same thing using Python interface

In [4]:
ds = gdal.Open(f)
projection = ds.GetProjectionRef()
#print(projection)
info =gdal.Info(ds)
#print(info)

#### Need some coordinates for the clipping operation

In [5]:
# This .csv file contains the corners of the rotated island grid
# It was created by running make_grid(**r), where r is the dict read in as r = yaml2dict('island_box.yml')
boxfn = 'ncorebx_refac.csv'

box = pd.read_csv(boxfn).values
print(box)
print(np.shape(box))
ulx = np.floor( box[2,0] )-2.
uly = np.ceil( box[1,1] )+2.
lrx = np.ceil( box[0,0] )+2.
lry = np.floor( box[3,1] )-2.
width = lrx-ulx
height = uly - lry
print('ulx, uly {:.2f}, {:.2f}'.format(ulx,uly))
print('lrx, lry {:.2f}, {:.2f}'.format(lrx,lry))
print('width, height {:.2f}, {:.2f}'.format(width, height))

[[ 405614.07999221 3880163.30413921]
 [ 404611.05321328 3881277.2782326 ]
 [ 377487.01022818 3856854.68023111]
 [ 378490.03700711 3855740.70613772]]
(4, 2)
ulx, uly 377485.00, 3881280.00
lrx, lry 405617.00, 3855738.00
width, height 28132.00, 25542.00


#### Andy Richie says:
This is what I use for all DEMs and orthos now:
 
`for %i in (.\*.tif) do gdal_translate %i .\cog\%~ni_cog.tif -of COG -stats -co BLOCKSIZE=256 -co COMPRESS=DEFLATE -co PREDICTOR=YES -co NUM_THREADS=ALL_CPUS -co BIGTIFF=YES`

I found that on my 6-cpu (12-thread) Win64 laptop with 32GB, and timing for NUM_THREADS = '1', '4', and  'ALL_CPUS' was 46.5, 23.1, and 22.9 s.

#### Using the subprocess.run() function, the arguments need to be broken down to pieces:

In [9]:
%%time
src_file = f
dest_file = 'clippedCOG2.tif'
p = subprocess.run(['gdal_translate','--version'],capture_output=True)
print(p.stderr.decode())
print(p.stdout.decode())

ulxs = '{:.1f}'.format(ulx)
ulys = '{:.1f}'.format(uly)
lrxs = '{:.1f}'.format(lrx)
lrys = '{:.1f}'.format(lry)

p = subprocess.run(['gdal_translate',\
                    '-projwin',ulxs,ulys,lrxs,lrys,\
                    '-of','COG',\
                    '-co','COMPRESS=DEFLATE',\
                    '-co','BIGTIFF=YES',\
                    '-co','PREDICTOR=YES',\
                    '-co','BLOCKSIZE=256',\
                    '-co','NUM_THREADS=ALL_CPUS',\
                    '-stats','-strict',\
                    src_file,\
                    dest_file],\
                   capture_output=True)
print(p.stderr.decode())
print(p.stdout.decode())


GDAL 3.4.0, released 2021/11/04


Input file size is 27617, 24245
0...10...20...30...40...50...60...70...80...90...100 - done.

Wall time: 22.9 s


In [7]:
info = gdal.Info(f)
print(info)
info = gdal.Info('clippedCOG2.tif')
print(info)

Driver: GTiff/GeoTIFF
Files: C:/crs/proj/2021_FloSupp_Release/clipfest/std_err/20190830_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83UTM18_NAVD88_1m_SErr_cog.tif
       C:/crs/proj/2021_FloSupp_Release/clipfest/std_err/20190830_Ocracoke_Inlet_to_Ophelia_Inlet_NAD83UTM18_NAVD88_1m_SErr_cog.tif.aux.xml
Size is 27617, 24245
Coordinate System is:
PROJCRS["NAD83(2011) / UTM zone 18N",
    BASEGEOGCRS["NAD83(2011)",
        DATUM["NAD83 (National Spatial Reference System 2011)",
            ELLIPSOID["GRS 1980",6378137,298.257222101,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",6318]],
    CONVERSION["UTM zone 18N",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",-75,
            ANGLEUNIT["degree",0.0174532